<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Mongo-request" data-toc-modified-id="Mongo-request-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Mongo request</a></span></li><li><span><a href="#Dataframe-cleaning" data-toc-modified-id="Dataframe-cleaning-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Dataframe cleaning</a></span></li><li><span><a href="#Map-visualization" data-toc-modified-id="Map-visualization-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Map visualization</a></span></li></ul></div>

# San Francisco videogame companies

Most of our teams are developers and designers, we want to make them comfortable so it is important to consider their preferences.
That's why we are going to place the office in a zone where there are other emergent videogame companies that have rised at least 1 milliondollars. Moreover, focusing on videogame companies instead of any tech companies, we ensure that they also have designing interests so if they organize talks on the subject, it would be easy for our designers to attend. 

We are going to use Mongodb "companies".

In [1]:

from pymongo import MongoClient
client = MongoClient("mongodb://localhost:27017/ironhack")
db = client.get_database()

In [12]:
import folium
from folium import Choropleth, Circle, Marker, Icon, Map
from folium.plugins import HeatMap, MarkerCluster

In [2]:
companies = client.companies.companies
offices = client.companies.offices

In [3]:
import pandas as pd

## Mongo request

In [4]:
query = {"category_code": "games_video"}
project = {"_id":0, "name":1,"offices.city":1, "founded_year":1}
result = db["companies"].find(query,project).sort("founded_year",-1)
x= list (result)
len(x)
x[:4]

[{'name': 'Fliggo',
  'founded_year': 2012,
  'offices': [{'city': 'San Francisco'}]},
 {'name': 'Social Gaming Network',
  'founded_year': 2011,
  'offices': [{'city': 'Los Angeles'}, {'city': 'Beverly Hills'}]},
 {'name': 'Streamzy', 'founded_year': 2011, 'offices': []},
 {'name': 'Fuzz', 'founded_year': 2011, 'offices': [{'city': 'Santa Clara'}]}]

On a first overview we can deduce that the newest video games companies have been founded before 2011 so we can´t consider them exactly startups but let's consider those that are relatively new, for example those founded since 2005.
We will take into account those that have raise at least 1 million dollars, so the tech developers are happy.

In [5]:
query = {"category_code": "games_video",  "founded_year": {"$gte" : 2005}, "offices.city":"San Francisco", "total_money_raised":{"$regex" : "M"}}
project = {"_id":0, "name":1,"offices":1, "total_money_raised":1, "founded_year":1}
result = db["companies"].find(query,project)
sf_videogames= list (result)
len(sf_videogames)

24

In [6]:
sf_videogames[0]

{'name': 'Kyte',
 'founded_year': 2006,
 'total_money_raised': '$23.4M',
 'offices': [{'description': None,
   'address1': '442 Post Street',
   'address2': '10th Floor',
   'zip_code': '94102',
   'city': 'San Francisco',
   'state_code': 'CA',
   'country_code': 'USA',
   'latitude': 37.788482,
   'longitude': -122.409173}]}

In [7]:
sf_df = pd.json_normalize(sf_videogames,"offices",["name", "founded_year", "total_money_raised"])
sf_df.head()

,description,address1,address2,zip_code,city,state_code,country_code,latitude,longitude,name,founded_year,total_money_raised
0,None,442 Post Street,10th Floor,94102,San Francisco,CA,USA,37.788482,-122.409173,Kyte,2006,$23.4M
1,San Francisco Office,410 Townsend,Suite 400,94107,San Francisco,CA,USA,37.392936,-122.079480,Ustream,2007,$60.1M
2,Los Angeles Office,6725 W Sunset Blvd,Suite 320,90028,Los Angeles,CA,USA,NaN,NaN,Ustream,2007,$60.1M
3,Budapest Office,,,,Budapest,None,HUN,NaN,NaN,Ustream,2007,$60.1M
4,HQ,2415 3rd St,,94107,San Francisco,CA,USA,37.757758,-122.388243,Revision3,2005,$9M


## Dataframe cleaning

Although we have already filtered the companies depending on if they had an office in San Francisco or not, those that have more than one offfice are apparing in our dataframe, so we need to drop them.

In [11]:
# Get indexes where name column doesn't have value 'San Francisco'
indexNames = sf_df[~(sf_df['city'] == 'San Francisco')].index 
# Delete these row indexes from dataFrame
sf_df.drop(indexNames , inplace=True)
sf_df = sf_df[sf_df['latitude'].notna()]
sf_df = sf_df[sf_df['longitude'].notna()]
sf_df.head()

,description,address1,address2,zip_code,city,state_code,country_code,latitude,longitude,name,founded_year,total_money_raised
0,None,442 Post Street,10th Floor,94102,San Francisco,CA,USA,37.788482,-122.409173,Kyte,2006,$23.4M
1,San Francisco Office,410 Townsend,Suite 400,94107,San Francisco,CA,USA,37.392936,-122.079480,Ustream,2007,$60.1M
4,HQ,2415 3rd St,,94107,San Francisco,CA,USA,37.757758,-122.388243,Revision3,2005,$9M
5,None,374 Brannan St.,None,94107,San Francisco,CA,USA,37.780716,-122.393913,CastTV,2006,$3.1M
6,San Francisco,60 Broadway,,94111,San Francisco,CA,USA,37.787092,-122.399972,Curse,2006,$12M


In [32]:
len(sf_df)

21

In [27]:
import geopandas as gpd

In [28]:
sf_gdf = gpd.GeoDataFrame(sf_df,geometry = gpd.points_from_xy(sf_df.longitude, sf_df.latitude))

In [33]:
sf_gdf.head()

,description,address1,address2,zip_code,city,state_code,country_code,latitude,longitude,name,founded_year,total_money_raised,geometry
0,None,442 Post Street,10th Floor,94102,San Francisco,CA,USA,37.788482,-122.409173,Kyte,2006,$23.4M,POINT (-122.40917 37.78848)
1,San Francisco Office,410 Townsend,Suite 400,94107,San Francisco,CA,USA,37.392936,-122.079480,Ustream,2007,$60.1M,POINT (-122.07948 37.39294)
4,HQ,2415 3rd St,,94107,San Francisco,CA,USA,37.757758,-122.388243,Revision3,2005,$9M,POINT (-122.38824 37.75776)
5,None,374 Brannan St.,None,94107,San Francisco,CA,USA,37.780716,-122.393913,CastTV,2006,$3.1M,POINT (-122.39391 37.78072)
6,San Francisco,60 Broadway,,94111,San Francisco,CA,USA,37.787092,-122.399972,Curse,2006,$12M,POINT (-122.39997 37.78709)


In [30]:
sf_gdf.to_csv("data/san_francisco_vg_comp_gdf.csv")

## Map visualization

In [22]:
inicial_lat = 37.7825098
inicial_long = -122.4077973

In [23]:
map_1 = folium.Map(location = [inicial_lat,inicial_long], zoom_start = 15)


In [21]:
HeatMap(data = sf_df [["latitude", "longitude"]], radius =15).add_to(map_1)
map_1

We can see that these companies tend to accumulate in three areas. Let's find out if we can satify more requirements near here.

In [24]:
#Van Ness avenue with Market Street
loc1 = 37.77501,-122.4167646
#2nd with Bryan Street more or les
loc2 =37.7832303,-122.3936295
#Motgomery St. Station
loc3 =37.7887223,-122.4009011